# Final Project

*Please fill out the relevant cells below according to the instructions. When done, save the notebook and export it to PDF, upload both the `ipynb` and the PDF file to Canvas.*

## Group Members

*Group submission is highly encouraged. If you submit as part of group, list all group members here. Groups can comprise up to 5 students.*

* Adam Applegate
* Beatrix Brahms
* 

---

## Sparse Interactions

### Preparation (3pts)

Review the paper [The Kernel Interaction Trick: Fast Bayesian Discovery of Pairwise Interactions in High Dimensions](https://arxiv.org/abs/1905.06501) by Agrawal et al. (2019). Start with the general concepts and then go into the finer details.

When you feel comfortable with the content, answer the following questions:

1. Why does the Gaussian scale mixture prior promote sparsity of the regression coefficients $\theta$?
2. What are the required properties of the model in Eq. (3) that allow it to be rewritten in the form of Eq. (6)?
3. What are the conceptual and practical limitation of the approach?

**Hint:** Some of the answers may require parsing the relevant references.

**1** Based on Griffin and Brown (2017), hierarchical priors in Bayesian regression are dependent on the size of the coefficients, where the hyperparameters control for shrinkage of coefficients similar to a penalty term. In particular, a small estimated variance in coefficients would enforce some shrinkage and likely yield a sparser model. The Gaussian scale mixture is in fact just a special case of hierarchical priors, in which $\tau$ and $\Sigma_\tau$ control the shrinkage of $\theta$. 

**2** This result is possible due to the Proposition 4.1 in Agrawal et al (2019). The function $\Phi_2$ is designed such that it can be rewritten as a GP, namely the GP $g = \theta^T\Phi_2$. Then, the equivalence is completed by using the covariance $k_\tau(x^{(i)},x^{(j})=\Phi_2(x^{(i)})\Sigma_\tau\Phi_2(x^{(j)})$ for $g\sim GP(0,k_\tau)$. Based on the weight-space view of GP from Rasmussen and Williams (2006), for any draw $g|\tau\sim N(0,k_\tau)$, there exists a parameter vector $\theta$ such that $g=\theta^T\Phi_2$. This completes the re-parametrization from Equation (3) to (6). 

**3** In the discussion of Priors for Related Predictors in Chipman (1996), one practical instance in which the strong hierarchy property would fail is in atmospheric sciences. A key relation is $log(Y)=log(A)+BC$, in which the interaction term $BC$ would not satisfy strong hierarchy, and thus could not be properly modeled using this approach. One conceptual limitation of the approach is the use of pairwise interactions. On one hand, the dimension of the parameter space is increased from $p$ to $\frac{p(p+1)}{2}$, which could be extremely computationally expensive when $p$ is not even that large; on the other hand, there may be instances that involve higher-order polynomial interactions, and thus this model of at-most pairwise interactions could be too limited in its scope. In addition, the runtime is cubic and memory is quadraic with respect to the sample size $N$, which is as good or worse than other sampling methods it is compared to (NAIVE, WOODBURY, and FULL)

### Code adaptation (3pts)

The method SKIM from chapter 6 has been implemented in jax/Numpyro [here](https://pyro.ai/numpyro/examples/sparse_regression.html). Review the code and recognize how the theoretical concepts of the Kernel Interaction Trick and the specific features of SKIM have been implemented. Then copy the code to this notebook and modify it so that you can execute the provided test example inline. Confirm that you get a result comparable to theirs.

The last step of their example analysis (sampling from the posterior with the method `sample_theta_space`) often returns `nan`s. It also reports the posterior for all $\theta$ (active and inactive ones), and only for one sample at a time. That's really clunky. Modify this function to produce flat posteriors samples from the MCMC (with an arbitrary length of samples) but only for the active direct and pairwise interaction terms. Visualize the posterior from the example with `corner`.

### Application (4pts)

COVID-19 dominates the news, with many countries still reporting rising case numbers. One surprising fact is that the mortality rate (i.e. the fraction of the infected who have died, also called **case-fatality ratio**) differs *a lot* between countries, from 15% to less than 1%. Find out which features (such as age distribution, population health indices, economic factors, COVID-specific factors ...) influences that rate.

This task has three parts:

* Think about what possible effects there could be.
* Find suitable data for as many countries as possible in public data archives. Combine them into a master data set.
* Perform the inference.

You will probably need to iterate and refine along the way. Explain your reasoning about the kinds of features you decided to include in your analysis. Then report the most important direct and pairwise interactions. Visualized the posterior samples with `corner`.

**Note:** This is an exploratory study. If your approach is sound, but the data don't show firm trends, partial points will be awarded. Include your final data compilation as a separate file with your submission.

**Hints:** 

* Start [here](https://coronavirus.jhu.edu/data/mortality).
* Don't forget to standardize the data by subtracting the mean and dividing by the standard deviation.

In [1]:
%matplotlib inline

In [2]:
import argparse
import itertools
import os
import time

import numpy as onp

import jax
from jax import vmap
import jax.numpy as np
import jax.random as random

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

import corner

def dot(X, Z):
    return np.dot(X, Z[..., None])[..., 0]


# The kernel that corresponds to our quadratic regressor. (According to prop 6.1)
def kernel(X, Z, eta1, eta2, c, jitter=1.0e-6):
    eta1sq = np.square(eta1)
    eta2sq = np.square(eta2)
    k1 = 0.5 * eta2sq * np.square(1.0 + dot(X, Z))
    k2 = -0.5 * eta2sq * dot(np.square(X), np.square(Z))
    k3 = (eta1sq - eta2sq) * dot(X, Z)
    k4 = np.square(c) - 0.5 * eta2sq
    if X.shape == Z.shape:
        k4 += jitter * np.eye(X.shape[0])
    return k1 + k2 + k3 + k4


# Most of the model code is concerned with constructing the sparsity inducing prior.
def model(X, Y, hypers):
    # Here X is the design matrix with N x p dimensions
    # read off dimensions P and N
    # S ??? - like a sparsity coeff? 
    S, P, N = hypers['expected_sparsity'], X.shape[1], X.shape[0]

    # sample variables from p. 18
    sigma = numpyro.sample("sigma", dist.HalfNormal(hypers['alpha3']))
    phi = sigma * (S / np.sqrt(N)) / (P - S)
    eta1 = numpyro.sample("eta1", dist.HalfCauchy(phi))

    msq = numpyro.sample("msq", dist.InverseGamma(hypers['alpha1'], hypers['beta1']))
    xisq = numpyro.sample("xisq", dist.InverseGamma(hypers['alpha2'], hypers['beta2']))

    eta2 = np.square(eta1) * np.sqrt(xisq) / msq

    lam = numpyro.sample("lambda", dist.HalfCauchy(np.ones(P)))
    kappa = np.sqrt(msq) * lam / np.sqrt(msq + np.square(eta1 * lam))

    # sample observation noise
    var_obs = numpyro.sample("var_obs", dist.InverseGamma(hypers['alpha_obs'], hypers['beta_obs']))

    # compute kernel (as in proposition 6.1)
    kX = kappa * X
    k = kernel(kX, kX, eta1, eta2, hypers['c']) + var_obs * np.eye(N)
    assert k.shape == (N, N)

    # sample Y according to the standard gaussian process formula
    numpyro.sample("Y", dist.MultivariateNormal(loc=np.zeros(X.shape[0]), covariance_matrix=k),
                   obs=Y)

    
    
# I guess broadly corresponds to Section 5 of the paper??

# Compute the mean and variance of coefficient theta_i (where i = dimension) for a
# MCMC sample of the kernel hyperparameters (eta1, xisq, ...).
# Compare to theorem 5.1 in reference [1].
def compute_singleton_mean_variance(X, Y, dimension, msq, lam, eta1, xisq, c, var_obs):
    P, N = X.shape[1], X.shape[0]

    probe = np.zeros((2, P))
    probe = jax.ops.index_update(probe, jax.ops.index[:, dimension], np.array([1.0, -1.0]))

    eta2 = np.square(eta1) * np.sqrt(xisq) / msq
    kappa = np.sqrt(msq) * lam / np.sqrt(msq + np.square(eta1 * lam))

    kX = kappa * X
    kprobe = kappa * probe

    k_xx = kernel(kX, kX, eta1, eta2, c) + var_obs * np.eye(N)
    k_xx_inv = np.linalg.inv(k_xx)
    k_probeX = kernel(kprobe, kX, eta1, eta2, c)
    k_prbprb = kernel(kprobe, kprobe, eta1, eta2, c)

    vec = np.array([0.50, -0.50]) ## a = (1/2, -1/2)
    mu = np.matmul(k_probeX, np.matmul(k_xx_inv, Y))
    mu = np.dot(mu, vec)

    var = k_prbprb - np.matmul(k_probeX, np.matmul(k_xx_inv, np.transpose(k_probeX)))
    var = np.matmul(var, vec)
    var = np.dot(var, vec)

    return mu, var


# Compute the mean and variance of coefficient theta_ij for a MCMC sample of the
# kernel hyperparameters (eta1, xisq, ...). Compare to theorem 5.1 in reference [1].
def compute_pairwise_mean_variance(X, Y, dim1, dim2, msq, lam, eta1, xisq, c, var_obs):
    # Here X is the design matrix with N x p dimensions
    # read off dimensions P and N
    P, N = X.shape[1], X.shape[0]

    probe = np.zeros((4, P))
    probe = jax.ops.index_update(probe, jax.ops.index[:, dim1], np.array([1.0, 1.0, -1.0, -1.0]))
    probe = jax.ops.index_update(probe, jax.ops.index[:, dim2], np.array([1.0, -1.0, 1.0, -1.0]))
    
    
    # compute eta2 and kappa from p. 18 

    eta2 = np.square(eta1) * np.sqrt(xisq) / msq
    kappa = np.sqrt(msq) * lam / np.sqrt(msq + np.square(eta1 * lam))

    kX = kappa * X
    kprobe = kappa * probe

    # ?? compute a bunch of matrices w/ kernels ??
    k_xx = kernel(kX, kX, eta1, eta2, c) + var_obs * np.eye(N)
    k_xx_inv = np.linalg.inv(k_xx)
    k_probeX = kernel(kprobe, kX, eta1, eta2, c)
    k_prbprb = kernel(kprobe, kprobe, eta1, eta2, c)

    vec = np.array([0.25, -0.25, -0.25, 0.25]) ## ?? not sure why not (-1/2, 1/2, -1, 1) ??
    mu = np.matmul(k_probeX, np.matmul(k_xx_inv, Y))
    mu = np.dot(mu, vec)

    var = k_prbprb - np.matmul(k_probeX, np.matmul(k_xx_inv, np.transpose(k_probeX)))
    var = np.matmul(var, vec)
    var = np.dot(var, vec)

    return mu, var


# Sample coefficients theta from the posterior for a given MCMC sample.
# The first P returned values are {theta_1, theta_2, ...., theta_P}, while
# the remaining values are {theta_ij} for i,j in the list `active_dims`,
# sorted so that i < j.
def sample_theta_space(X, Y, active_dims, msq, lam, eta1, xisq, c, var_obs): #(section B.5) ?
    # Here X is the design matrix with N x p dimensions
    # read off dimensions P and N
    # and number of active dimensions? 
    P, N, M = X.shape[1], X.shape[0], len(active_dims)
    
    # the total number of coefficients we return
    num_coefficients = P + M * (M - 1) // 2

    probe = np.zeros((2 * P + 2 * M * (M - 1), P))
    vec = np.zeros((num_coefficients, 2 * P + 2 * M * (M - 1)))
    start1 = 0
    start2 = 0

    for dim in range(P):
        probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 2, dim], np.array([1.0, -1.0]))
        vec = jax.ops.index_update(vec, jax.ops.index[start2, start1:start1 + 2], np.array([0.5, -0.5]))
        start1 += 2
        start2 += 1

    for dim1 in active_dims:
        for dim2 in active_dims:
            if dim1 >= dim2:
                continue
            probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 4, dim1],
                                         np.array([1.0, 1.0, -1.0, -1.0]))
            probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 4, dim2],
                                         np.array([1.0, -1.0, 1.0, -1.0]))
            vec = jax.ops.index_update(vec, jax.ops.index[start2, start1:start1 + 4],
                                       np.array([0.25, -0.25, -0.25, 0.25]))
            start1 += 4
            start2 += 1

    eta2 = np.square(eta1) * np.sqrt(xisq) / msq
    kappa = np.sqrt(msq) * lam / np.sqrt(msq + np.square(eta1 * lam))

    kX = kappa * X
    kprobe = kappa * probe

    k_xx = kernel(kX, kX, eta1, eta2, c) + var_obs * np.eye(N)
    k_xx_inv = np.linalg.inv(k_xx)
    k_probeX = kernel(kprobe, kX, eta1, eta2, c)
    k_prbprb = kernel(kprobe, kprobe, eta1, eta2, c)

    mu = np.matmul(k_probeX, np.matmul(k_xx_inv, Y))
    mu = np.sum(mu * vec, axis=-1)

    covar = k_prbprb - np.matmul(k_probeX, np.matmul(k_xx_inv, np.transpose(k_probeX)))
    covar = np.matmul(vec, np.matmul(covar, np.transpose(vec)))
    L = np.linalg.cholesky(covar)

    # sample from N(mu, covar)
    sample = mu + np.matmul(L, onp.random.randn(num_coefficients))

    return sample


# Helper function for doing HMC inference
def run_inference(model, args, rng_key, X, Y, hypers):
    start = time.time()
    kernel = NUTS(model)
    mcmc = MCMC(kernel, args.num_warmup, args.num_samples, num_chains=args.num_chains,
                progress_bar=False if "NUMPYRO_SPHINXBUILD" in os.environ else True)
    mcmc.run(rng_key, X, Y, hypers)
    mcmc.print_summary()
    print('\nMCMC elapsed time:', time.time() - start)
    return mcmc.get_samples()


# Get the mean and variance of a gaussian mixture
def gaussian_mixture_stats(mus, variances):
    mean_mu = np.mean(mus)
    mean_var = np.mean(variances) + np.mean(np.square(mus)) - np.square(mean_mu)
    return mean_mu, mean_var


# Create artificial regression dataset where only S out of P feature
# dimensions contain signal and where there is a single pairwise interaction
# between the first and second dimensions.
def get_data(N=20, S=2, P=10, sigma_obs=0.05):
    assert S < P and P > 1 and S > 0
    onp.random.seed(0)

    X = onp.random.randn(N, P)
    # generate S coefficients with non-negligible magnitude
    W = 0.5 + 2.5 * onp.random.rand(S)
    # generate data using the S coefficients and a single pairwise interaction
    Y = onp.sum(X[:, 0:S] * W, axis=-1) + X[:, 0] * X[:, 1] + sigma_obs * onp.random.randn(N)
    Y -= np.mean(Y)
    Y_std = np.std(Y)

    assert X.shape == (N, P)
    assert Y.shape == (N,)

    return X, Y / Y_std, W / Y_std, 1.0 / Y_std


# Helper function for analyzing the posterior statistics for coefficient theta_i
def analyze_dimension(samples, X, Y, dimension, hypers):
    vmap_args = (samples['msq'], samples['lambda'], samples['eta1'], samples['xisq'], samples['var_obs'])
    mus, variances = vmap(lambda msq, lam, eta1, xisq, var_obs:
                          compute_singleton_mean_variance(X, Y, dimension, msq, lam,
                                                          eta1, xisq, hypers['c'], var_obs))(*vmap_args)
    mean, variance = gaussian_mixture_stats(mus, variances)
    std = np.sqrt(variance)
    return mean, std


# Helper function for analyzing the posterior statistics for coefficient theta_ij
def analyze_pair_of_dimensions(samples, X, Y, dim1, dim2, hypers):
    vmap_args = (samples['msq'], samples['lambda'], samples['eta1'], samples['xisq'], samples['var_obs'])
    mus, variances = vmap(lambda msq, lam, eta1, xisq, var_obs:
                          compute_pairwise_mean_variance(X, Y, dim1, dim2, msq, lam,
                                                         eta1, xisq, hypers['c'], var_obs))(*vmap_args)
    mean, variance = gaussian_mixture_stats(mus, variances)
    std = np.sqrt(variance)
    return mean, std

In [3]:
def sample_theta_space_trial(X, Y, active_dims, msq, lam, eta1, xisq, c, var_obs, N_samps, dim_pair_arr): #(section B.5) ?
    # Here X is the design matrix with N x p dimensions
    # read off dimensions P and N
    # and number of active dimensions? 
    P, N, M = X.shape[1], X.shape[0], len(active_dims)
    
    # the total number of coefficients we return
    num_coefficients = P + M * (M - 1) // 2

    probe = np.zeros((2 * P + 2 * M * (M - 1), P))
    vec = np.zeros((num_coefficients, 2 * P + 2 * M * (M - 1)))
    start1 = 0
    start2 = 0

    for dim in range(P):
        probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 2, dim], np.array([1.0, -1.0]))
        vec = jax.ops.index_update(vec, jax.ops.index[start2, start1:start1 + 2], np.array([0.5, -0.5]))
        start1 += 2
        start2 += 1

    for dim1 in active_dims:
        for dim2 in active_dims:
            if dim1 >= dim2:
                continue
            probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 4, dim1],
                                         np.array([1.0, 1.0, -1.0, -1.0]))
            probe = jax.ops.index_update(probe, jax.ops.index[start1:start1 + 4, dim2],
                                         np.array([1.0, -1.0, 1.0, -1.0]))
            vec = jax.ops.index_update(vec, jax.ops.index[start2, start1:start1 + 4],
                                       np.array([0.25, -0.25, -0.25, 0.25]))
            start1 += 4
            start2 += 1

    eta2 = np.square(eta1) * np.sqrt(xisq) / msq
    kappa = np.sqrt(msq) * lam / np.sqrt(msq + np.square(eta1 * lam))

    kX = kappa * X
    kprobe = kappa * probe

    k_xx = kernel(kX, kX, eta1, eta2, c) + var_obs * np.eye(N)
    k_xx_inv = np.linalg.inv(k_xx)
    k_probeX = kernel(kprobe, kX, eta1, eta2, c)
    k_prbprb = kernel(kprobe, kprobe, eta1, eta2, c)

    mu = np.matmul(k_probeX, np.matmul(k_xx_inv, Y))
    mu = np.sum(mu * vec, axis=-1)

    covar = k_prbprb - np.matmul(k_probeX, np.matmul(k_xx_inv, np.transpose(k_probeX)))
    covar = np.matmul(vec, np.matmul(covar, np.transpose(vec)))
    L = np.linalg.cholesky(covar)
    
    #print("mu" + str(mu))
    #print("cov" + str(covar))
    
    # sample from N(mu, covar)
    sample = mu + np.matmul(L, onp.random.randn(num_coefficients))
    
        ####### ~~~~~~~~~~~~~ CHANGES~~~~~~~~~~~~~~~~~~~
    print("NUM of DIM: " + str(P))
    print("Num of all Thetas: " + str(len(mu)))
    # sample from active dims only? 
    all_active_dims = active_dims + dim_pair_arr
    mu_active = np.array([mu[i] for i in all_active_dims])
    
    cov_active = []
    for j in all_active_dims:
        cov_act_j = [covar[j][i] for i in all_active_dims]
        #print(cov_act_j)
        cov_active.append(cov_act_j)
    cov_active = onp.array(cov_active)
    
    print("mu_act" + str(mu_active))
    print("cov_act" + str(cov_active))
    rng_key = random.PRNGKey(0)
    samps = numpyro.distributions.MultivariateNormal(loc = onp.array(mu_active), 
                                                     covariance_matrix = cov_active).sample(rng_key, sample_shape = (1, N_samps))

    samps_new = onp.reshape(samps, (N_samps, len(all_active_dims)))
    
    return samps_new

In [4]:
def main(args, hypers):
    #X, Y, expected_thetas, expected_pairwise = get_data(N=args.num_data, P=args.num_dimensions,
    #                                                    S=args.active_dimensions)
    
    Y = np.array(df.iloc[:,0])
    X = np.array(df.iloc[:,1:])

    # do inference
    rng_key = random.PRNGKey(0)
    samples = run_inference(model, args, rng_key, X, Y, hypers)

    # compute the mean and square root variance of each coefficient theta_i
    means, stds = vmap(lambda dim: analyze_dimension(samples, X, Y, dim, hypers))(np.arange(args.num_dimensions))
    num_dims = len(means)
    #print("Coefficients theta_1 to theta_%d used to generate the data:" % args.active_dimensions, expected_thetas)
    #print("The single quadratic coefficient theta_{1,2} used to generate the data:", expected_pairwise)
    active_dimensions = []

    for dim, (mean, std) in enumerate(zip(means, stds)):
        # we mark the dimension as inactive if the interval [mean - 3 * std, mean + 3 * std] contains zero
        lower, upper = mean - 1.0 * std, mean + 1.0 * std
        inactive = "inactive" if lower < 0.0 and upper > 0.0 else "active"
        if inactive == "active":
            active_dimensions.append(dim)
        print("[dimension %02d/%02d]  %s:\t%.2e +- %.2e" % (dim + 1, args.num_dimensions, inactive, mean, std))

    print("Identified a total of %d active dimensions; expected %d." % (len(active_dimensions),
                                                                        args.active_dimensions))

    # Compute the mean and square root variance of coefficients theta_ij for i,j active dimensions.
    # Note that the resulting numbers are only meaningful for i != j.
    if len(active_dimensions) > 0:
        dim_pairs = np.array(list(itertools.product(active_dimensions, active_dimensions)))
        means, stds = vmap(lambda dim_pair: analyze_pair_of_dimensions(samples, X, Y,
                                                                       dim_pair[0], dim_pair[1], hypers))(dim_pairs)
        # print(dim_pairs)
        dim_pair_arr = []
        dim_pair_index = num_dims -1
        dim_pair_name = []
        for dim_pair, mean, std in zip(dim_pairs, means, stds):
            dim1, dim2 = dim_pair
            if dim1 >= dim2:
                continue
            dim_pair_index += 1  
            lower, upper = mean - 3.0 * std, mean + 3.0 * std
            if not (lower < 0.0 and upper > 0.0):
                dim_pair_arr.append(dim_pair_index)
                dim_pair_name.append('%d and %d'%(dim1 + 1, dim2 + 1))
                format_str = "Identified pairwise interaction between dimensions %d and %d: %.2e +- %.2e"
                print(format_str % (dim1 + 1, dim2 + 1, mean, std))
        print(dim_pair_arr)
        print(dim_pair_name)
        # Draw a single sample of coefficients theta from the posterior, where we return all singleton
        # coefficients theta_i and pairwise coefficients theta_ij for i, j active dimensions. We use the
        # final MCMC sample obtained from the HMC sampler.
        N_samps = 100
        thetas = sample_theta_space_trial(X, Y, active_dimensions, samples['msq'][-1], samples['lambda'][-1],
                                            samples['eta1'][-1], samples['xisq'][-1], hypers['c'], 
                                            samples['var_obs'][-1], N_samps, dim_pair_arr)
        print("Active_dimensions: " + str(active_dimensions))
        #print("Single posterior sample theta:\n", thetas)
        all_active_dimensions = active_dimensions + dim_pair_arr
        labels = ['dim '+str(i) for i in active_dimensions]
        for n in range(len(dim_pair_name)):
            labels.append('dim ' + dim_pair_name[n])
        fig = corner.corner(thetas, labels = labels);
        fig.show()

# TEST ON COVID DATA

In [56]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

raw_data = pd.read_csv('raw_data.csv')
df = raw_data.copy()

country = df[df.columns[[0]]] # save countries
df.drop(df.columns[0], axis=1, inplace=True) # drop countries
df[df.columns] = StandardScaler().fit_transform(df) # scale
# df.iloc[:,0] = StandardScaler().fit_transform(df.iloc[:,0].to_numpy().reshape(-1,1)) # scale

# df = df.loc[:,['Population median age (years)', 'Cases per 1M population', 'diabetes_prevalence', 'Tests/ 1M pop']]
# df = df.iloc[:30,:]

df


,Case-Fatality,Human Development Index (HDI),Population median age (years),Adult mortality rate,Life expectancy at birth (years),Mortality rate attributed to exposure to unsafe WASH services (per 100 000 population) (SDG 3.9.2),Population proportion over 60 (%),Current health expenditure (CHE) per capita in US$,UHC index of service coverage (SCI),Cases per 1M population,Urban population (% of total population),diabetes_prevalence,hospital_beds_per_100k,cvd_death_rate,GDP per capita,Tests/ 1M pop
0,0.191215,1.203150,0.865347,-0.415850,0.714440,-0.576234,1.790465,3.962344,1.091062,2.754711,0.907303,0.961667,-0.023778,-0.742498,2.019030,0.671197
1,2.534257,1.203150,1.175225,-1.051761,1.120574,-0.576234,0.119199,1.100189,1.288382,2.073479,0.959909,-0.893201,-0.114475,-0.986940,1.225108,0.499712
2,2.225284,0.961579,1.628975,-1.227652,1.316638,-0.581976,0.171357,0.643761,0.959516,2.566817,0.362907,-0.750738,0.137898,-1.062809,1.013720,1.474719
3,2.534257,1.013811,1.219493,-0.997641,1.330643,-0.570492,0.145833,1.333646,0.696423,1.106240,0.823833,-0.753587,1.242031,-1.291539,1.178186,0.306799
4,2.765986,1.196621,1.330164,-0.984111,1.092565,-0.570492,-0.291404,1.390868,1.091062,3.347087,1.632596,-0.890352,1.107958,-1.048059,1.460354,1.621932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,-1.018928,0.289098,0.201323,-0.185839,0.266293,-0.518813,-0.284746,-0.557400,-0.092856,-0.695878,-2.030720,0.930325,0.303518,-0.354086,-0.470704,-0.539635
64,1.710330,-1.127683,-1.115658,2.560758,-1.680346,0.824832,-0.357988,-0.579536,-0.882135,-0.726423,-1.398110,-1.594119,-0.445715,0.581003,-1.022397,-0.586538
65,-0.606965,-1.734876,-1.259530,1.004804,-1.106157,1.921567,-0.208174,-0.617534,-1.868734,-0.726423,-1.925370,0.015713,-0.997781,-0.476164,-1.050487,-0.602008
66,0.551683,-1.636941,-1.370201,1.491885,-1.288217,1.037288,-0.355769,-0.614397,-1.408321,-0.726423,-2.101614,-0.990076,-0.603448,-0.098634,-1.087411,-0.611902


In [29]:
# df.corr()

In [57]:
# setup hyperparameters
# hypers = {'expected_sparsity': max(1.0, args.num_dimensions / 10),
#           'alpha1': 3.0, 'beta1': 1.0,
#           'alpha2': 3.0, 'beta2': 1.0,
#           'alpha3': 1.0, 'c': 1.0,
#           'alpha_obs': 3.0, 'beta_obs': 1.0}

hypers = {'expected_sparsity': max(1.0, args.num_dimensions / 2),
          'alpha1': 0.26872577050471647, 
          'alpha2': 4.818866884657901, 
          'alpha3': 6.812836016637205, 
          'alpha_obs': 49.093191654133754, 
          'beta1': 1.2942745182532156, 
          'beta2': 0.21267247881371867, 
          'beta_obs': 3.3406960438157594, 
          'c': 2.751017838090896}

In [58]:
if __name__ == "__main__":
    assert numpyro.__version__.startswith('0.2.4')
    parser = argparse.ArgumentParser(description="Gaussian Process example")
#     parser.add_argument("-n", "--num-samples", nargs="?", default=1000, type=int)
#     parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
#     parser.add_argument("--num-chains", nargs='?', default=1, type=int)
#     parser.add_argument("--num-data", nargs='?', default=100, type=int)
#     parser.add_argument("--num-dimensions", nargs='?', default=20, type=int)
#     parser.add_argument("--active-dimensions", nargs='?', default=3, type=int)
#     parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')
    
    parser.add_argument("-n", "--num-samples", nargs="?", default=5000, type=int)
    parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
    parser.add_argument("--num-chains", nargs='?', default=1, type=int)
    parser.add_argument("--num-data", nargs='?', default=68, type=int)
    parser.add_argument("--num-dimensions", nargs='?', default=16, type=int)
    parser.add_argument("--active-dimensions", nargs='?', default=8, type=int)
    parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')

    #args = parser.parse_args() 
    args = parser.parse_args(args=[])  #68, 50, 5, 68, 16, 3, 'cpu'

    numpyro.set_platform(args.device)
    numpyro.set_host_device_count(args.num_chains)

    main(args, hypers)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Bonus challenge (2pts extra):

Find another application (e.g. from your area of research) where the kernel-interaction method is directly applicable, or could be applied with some modification. Describe the application for a statistically knowledgeable but non-expert audience (think: your peers in SML 515). In particular, explain why the sparse interaction ansatz is justified. Then demonstrate the use with a suitable data set of your own choice.

**Note:** If your description is convincing, but you don't find any data or it doesn't lead to conclusive results, partial points will be awarded. Make sure that you have permission to use the data and include it as separate file in your submission.

In [14]:
import numpy as onp
import pandas as pd
from sklearn.preprocessing import StandardScaler

# fxdata3 = pd.read_csv('fxdata3.csv')
# fxdata4 = pd.read_csv('fxdata4.csv')
fxdata5 = pd.read_csv('fxdata5.csv', header=0)
# fxdata10 = pd.read_csv('fxdata10.csv')

df = fxdata5.copy()
target_col = 'EURGBP_log_RealVol'
df['Target_' + target_col] = df[target_col].copy()
cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index('Target_' + target_col)))
df = df[cols].set_index('Date')


df = df.iloc[-252:,:]
df[::5].shape

(51, 245)

In [13]:
for c in df.columns[1:]:
    df[c] = (df[c] - df[c].mean()) / df[c].std()
    
df.shape

(252, 245)

In [32]:
hypers = {'expected_sparsity': max(1.0, args.num_dimensions / 10),
          'alpha1': 0.27126163430537936, 
            'alpha2': 8.54749838434074, 
         'alpha3': 0.903895331940715, 
         'alpha_obs': 1.5417239333242043, 
         'beta1': 0.8507152602802248, 
         'beta2': 0.10195933547631165, 
         'beta_obs': 0.1328785334645342, 
         'c': 0.1529779179024281}



In [11]:
param_grid = {'expected_sparsity': max(1.0, args.num_dimensions // 10),
              'alpha1': list(range(1,4)), 
              'alpha2': list(range(1,4)), 
              'alpha3': list(range(1,4)), 
              'alpha_obs': list(range(10,50,10)), 
              'beta1': list(range(1,4)), 
              'beta2': list(range(1,4)), 
              'beta_obs': list(range(1,4)), 
              'c': list(range(2))}

n_params = len(param_grid)
param_names = [_.lower() for _ in list(param_grid.keys())]
param_values = list(param_grid.values())
mdls = pd.DataFrame(onp.stack(onp.meshgrid(*param_values)).T.reshape(-1,n_params), columns=param_names)

In [53]:
hypers

{'expected_sparsity': 24,
 'alpha1': 1,
 'alpha2': 1,
 'alpha3': 1,
 'alpha_obs': 10,
 'beta1': 1,
 'beta2': 1,
 'beta_obs': 1,
 'c': 0}

In [ ]:
for i, mdl in mdls.iterrows():
    
    hypers = mdl.to_dict()

    try:
        if __name__ == "__main__":
            assert numpyro.__version__.startswith('0.2.4')
            parser = argparse.ArgumentParser(description="Gaussian Process example")
        #     parser.add_argument("-n", "--num-samples", nargs="?", default=1000, type=int)
        #     parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
        #     parser.add_argument("--num-chains", nargs='?', default=1, type=int)
        #     parser.add_argument("--num-data", nargs='?', default=100, type=int)
        #     parser.add_argument("--num-dimensions", nargs='?', default=20, type=int)
        #     parser.add_argument("--active-dimensions", nargs='?', default=3, type=int)
        #     parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')

            parser.add_argument("-n", "--num-samples", nargs="?", default=1000, type=int)
            parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
            parser.add_argument("--num-chains", nargs='?', default=1, type=int)
            parser.add_argument("--num-data", nargs='?', default=252, type=int)
            parser.add_argument("--num-dimensions", nargs='?', default=245, type=int)
            parser.add_argument("--active-dimensions", nargs='?', default=25, type=int)
            parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')

            #args = parser.parse_args() 
            args = parser.parse_args(args=[])

            numpyro.set_platform(args.device)
            numpyro.set_host_device_count(args.num_chains)

            main(args, hypers)

    except RuntimeError:
        continue

In [10]:
# hypers = {'expected_sparsity': max(1.0, args.num_dimensions // 10),
#  'alpha1': 1,
#  'alpha2': 1,
#  'alpha3': 1,
#  'alpha_obs': 10,
#  'beta1': 1,
#  'beta2': 1,
#  'beta_obs': 1,
#  'c': 0}

# if __name__ == "__main__":
#             assert numpyro.__version__.startswith('0.2.4')
#             parser = argparse.ArgumentParser(description="Gaussian Process example")
#         #     parser.add_argument("-n", "--num-samples", nargs="?", default=1000, type=int)
#         #     parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
#         #     parser.add_argument("--num-chains", nargs='?', default=1, type=int)
#         #     parser.add_argument("--num-data", nargs='?', default=100, type=int)
#         #     parser.add_argument("--num-dimensions", nargs='?', default=20, type=int)
#         #     parser.add_argument("--active-dimensions", nargs='?', default=3, type=int)
#         #     parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')

#             parser.add_argument("-n", "--num-samples", nargs="?", default=1000, type=int)
#             parser.add_argument("--num-warmup", nargs='?', default=500, type=int)
#             parser.add_argument("--num-chains", nargs='?', default=1, type=int)
#             parser.add_argument("--num-data", nargs='?', default=252, type=int)
#             parser.add_argument("--num-dimensions", nargs='?', default=245, type=int)
#             parser.add_argument("--active-dimensions", nargs='?', default=25, type=int)
#             parser.add_argument("--device", default='cpu', type=str, help='use "cpu" or "gpu".')

#             #args = parser.parse_args() 
#             args = parser.parse_args(args=[])

#             numpyro.set_platform(args.device)
#             numpyro.set_host_device_count(args.num_chains)

#             main(args, hypers)